In [1]:
import os
from sqlalchemy import create_engine

db_config = {
    "user": "postgres",
    "password": "password",
    "host": "localhost", 
    "port": "5432", 
    "database": "hris" 
}

# Construct the database URL
database_url = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"


if database_url is None:
    raise Exception("DATABASE_URL environment variable not set")

engine = create_engine(database_url)

In [2]:
tables_context = [
    {
        "table_name": "employee",
        "context": "List of employee in the app, contains first_name, last_name, email, gender, role, company, working_hours (weekly), and progress.",
    },
    {
        "table_name": "absensi_harian",
        "context": "Records daily attendance information for employees, capturing absence statuses. Contains employee_id, date, month, year, status (mean ontime or tardiness), overtime_hours, work_hours per day.",
    },
]

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core import SQLDatabase
import openai

OPENAI_API_KEY = ""


openai.api_key = OPENAI_API_KEY
llm = OpenAI(api_key=OPENAI_API_KEY, model="ft:gpt-3.5-turbo-0125:personal::9luaAwx2")

sql_database = SQLDatabase(
    engine, include_tables=[table["table_name"] for table in tables_context]
)

In [4]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex, SQLTableNodeMapping, SQLTableSchema
import pandas as pd

# list all the tables from database and crate table schema for prompt to LLM
tables = list(sql_database._all_tables)
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = []
for idx, table in enumerate(tables):
    table_schema_objs.append((SQLTableSchema(table_name = table, context_str = tables_context[idx]["context"])))

In [5]:
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine

# create a object index to store vectors 
obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex
)

# initializing query engine 
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=3), 
)

# Ask query to query engine and store it in a response variable
response = query_engine.query("Ganti nama dimas menjadi dimasa")

In [6]:
print(response)

The name "dimas" has been successfully changed to "dimasa" in the employee database.


In [13]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine(database_url)
Session = sessionmaker(bind=engine)
session = Session()

In [19]:
def add_or_update_employee(employee_data):
    prompt = f"""
    Given the following table schema: {table_schema_objs[0].context_str}
    Write an SQL query to insert or update the employee data. Based on this information {employee_data}
    """
    query_engine = SQLTableRetrieverQueryEngine(
        sql_database, obj_index.as_retriever(similarity_top_k=3), 
    )

    # Ask query to query engine and store it in a response variable
    response = query_engine.query(prompt)
    print(response)
    
    # try:
    #     session.execute(sql_query)
    #     session.commit()
    #     print("Employee data added/updated successfully.")
    # except Exception as e:
    #     session.rollback()
    #     print("Error:", str(e))

# Example employee data to be added or updated

employee_data = {
    "first_name": "John",
    "last_name": "Doe",
    "email": "john.doe@example.com",
    "gender": "Male",
    "role": "Engineer",
    "working_hours": 40
}

add_or_update_employee(employee_data)

Generated SQL Query: The SQL query provided is invalid. The correct query to insert or update the employee data based on the given information should be:

INSERT INTO employee (first_name, last_name, email, gender, role, working_hours)
VALUES ('John', 'Doe', 'john.doe@example.com', 'Male', 'Engineer', 40)
ON DUPLICATE KEY UPDATE
    first_name = VALUES(first_name),
    last_name = VALUES(last_name),
    gender = VALUES(gender),
    role = VALUES(role),
    working_hours = VALUES(working_hours);
